In [1]:
from rdflib import Graph,URIRef,Namespace,BNode,Literal
from rdflib.namespace import XSD,RDF
import pandas as pd
import argparse
from collections import defaultdict
from datetime import datetime
import json
from rdflib import URIRef


In [2]:
graph = Graph()
directory = 'sampleTSS.ttl'
print("Started loading graph...")
graph.parse(directory, format="turtle",publicID="https://example.org/")
print("Graph loaded successfully.")


Started loading graph...
Graph loaded successfully.


In [3]:
#Get unique time series snippets
snippet_set = set()

get_snippet_query = '''
        PREFIX sosa: <http://www.w3.org/ns/sosa/>
        PREFIX tss:  <https://w3id.org/tss#>
        PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>

        SELECT DISTINCT ?snippet
        WHERE{
            ?snippet a tss:Snippet
        }
        '''
print('Started identifying unique sensors')
for snippet in graph.query(get_snippet_query):
    snippet_term = snippet[0]   # this is an rdflib term (URIRef or Literal)
    snippet_set.add(snippet_term)

print('Snippets identified successfully')

Started identifying unique sensors
Snippets identified successfully


In [6]:
print(snippet_set)

{rdflib.term.URIRef('https://example.org/snippet/2026-01-05-press-1'), rdflib.term.URIRef('https://example.org/snippet/2026-01-04-cond-1'), rdflib.term.URIRef('https://example.org/snippet/2026-01-07-motion-1'), rdflib.term.URIRef('https://example.org/snippet/2026-01-09-noise-1'), rdflib.term.URIRef('https://example.org/snippet/2026-01-13-batt-1'), rdflib.term.URIRef('https://example.org/snippet/2026-01-10-turbidity-1'), rdflib.term.URIRef('https://example.org/snippet/2026-01-14-orp-1'), rdflib.term.URIRef('https://example.org/snippet/2026-01-01-temp-1'), rdflib.term.URIRef('https://example.org/snippet/2026-01-08-co2-1'), rdflib.term.URIRef('https://example.org/snippet/2026-01-12-level-1'), rdflib.term.URIRef('https://example.org/snippet/2026-01-06-light-1'), rdflib.term.URIRef('https://example.org/snippet/2026-01-03-ph-1'), rdflib.term.URIRef('https://example.org/snippet/2026-01-11-flow-1'), rdflib.term.URIRef('https://example.org/snippet/2026-01-15-custom-1'), rdflib.term.URIRef('http

In [4]:
#Create RDF
prefix_tss = Namespace('https://w3id.org/tss#')
prefix_ex  = Namespace('http://example.org/')
prefix_sosa  = Namespace('http://www.w3.org/ns/sosa/')
prefix_xsd  = Namespace('http://www.w3.org/2001/XMLSchema#')
final_graph = Graph()
final_graph.bind('tss', prefix_tss)
final_graph.bind('ex', prefix_ex)
final_graph.bind('sosa', prefix_sosa)
final_graph.bind('xsd', prefix_xsd)
print('Started creating final graph')

for snippet in snippet_set:
    # ensure snippet is a bare URI string (no angle brackets) before inserting
    snippet_uri = str(snippet)
    rdf_query = f'''
    PREFIX sosa: <http://www.w3.org/ns/sosa/>
    PREFIX tss:  <https://w3id.org/tss#>
    PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>

    CONSTRUCT {{
      ?s ?p ?o .
    }}
    WHERE {{
      VALUES ?s {{ <{snippet_uri}> }}
      ?s ?p ?o .
    }}
    '''
    constructed = graph.query(rdf_query)   # a Graph-like result for CONSTRUCT
    for triple in constructed:
        final_graph.add(triple)

Started creating final graph


In [5]:
print('Started writing file to disk')
final_graph.serialize('generated_TSS2RDF.ttl', format="turtle")
print('File written successfully')

Started writing file to disk
File written successfully
